In [ ]:
from PIL import Image, ImageDraw
import math
import numpy as np

# Create video frames
width = 800
height = 600
frames = []
ball_radius = 10
origin = (width // 2, height // 2)
center_x = width // 2
center_y = height // 2

angle = 0
radius = 0

def func(t):
    radius = 0.5 * t
    angle = 0.1 * t
    x, y = origin
    return (x + (radius * math.cos(angle)),
            y + (radius * math.sin(angle)))

# Generate frames
for _ in range(1000):
    # Create new frame
    image = Image.new('RGB', (width, height), 'white')
    draw = ImageDraw.Draw(image)
    
    # Calculate position
    x, y = func(_)
    #x = center_x + radius * math.cos(angle)
    #y = center_y + radius * math.sin(angle)
    
    # Draw ball
    draw.ellipse([x - ball_radius, y - ball_radius, 
                  x + ball_radius, y + ball_radius], fill='black')
    
    frames.append(image)
    
    # Update position
    angle += 0.1
    radius += 0.5

# Save as animated GIF
frames[0].save('spiral.gif', save_all=True, append_images=frames[1:], 
               duration=20, loop=0)

In [12]:
import cv2
import numpy as np
from PIL import Image
import glob

def process_video(gif_path):
    # Read the gif
    gif = Image.open(gif_path)
    frames = []
    try:
        while True:
            frame = gif.copy()
            # Convert PIL image to RGB before converting to numpy array
            frame = frame.convert('RGB')
            frames.append(np.array(frame))
            gif.seek(gif.tell() + 1)
    except EOFError:
        pass

    # Track ball in each frame
    tracking_data = []
    for frame_idx, frame in enumerate(frames):
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        
        # Threshold to isolate black ball
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contours[0])
            tracking_data.append({
                'frame': frame_idx,
                'time': frame_idx * 0.02,  # 20ms per frame
                'bbox': [x, y, x+w, y+h],
                'center': [x + w//2, y + h//2]
            })

    return tracking_data

# Example usage
tracking_results = process_video('spiral.gif')
for data in tracking_results:
    print(f"Frame {data['frame']}, Time: {data['time']:.2f}s, BBox: {data['bbox']}, Center: {data['center']}")

Frame 0, Time: 0.00s, BBox: [390, 290, 411, 311], Center: [400, 300]
Frame 1, Time: 0.02s, BBox: [391, 290, 412, 311], Center: [401, 300]
Frame 2, Time: 0.04s, BBox: [392, 291, 413, 312], Center: [402, 301]
Frame 3, Time: 0.06s, BBox: [392, 292, 413, 313], Center: [402, 302]
Frame 4, Time: 0.08s, BBox: [392, 293, 413, 314], Center: [402, 303]
Frame 5, Time: 0.10s, BBox: [392, 294, 413, 315], Center: [402, 304]
Frame 6, Time: 0.12s, BBox: [392, 295, 413, 316], Center: [402, 305]
Frame 7, Time: 0.14s, BBox: [391, 296, 412, 317], Center: [401, 306]
Frame 8, Time: 0.16s, BBox: [390, 297, 411, 318], Center: [400, 307]
Frame 9, Time: 0.18s, BBox: [389, 297, 410, 318], Center: [399, 307]
Frame 10, Time: 0.20s, BBox: [388, 298, 409, 319], Center: [398, 308]
Frame 11, Time: 0.22s, BBox: [387, 298, 408, 319], Center: [397, 308]
Frame 12, Time: 0.24s, BBox: [386, 298, 407, 319], Center: [396, 308]
Frame 13, Time: 0.26s, BBox: [385, 299, 406, 320], Center: [395, 309]
Frame 14, Time: 0.28s, BBox: [